In [1]:
from pulp import *
from pulp import LpProblem, LpVariable, LpMinimize, LpInteger, lpSum, value, LpBinary,LpStatusOptimal
import pulp
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore", message="Overwriting previously set objective.")
import utility
import docplex.mp.model
import docplex
import docplex_explainer
import mymetrics
import joblib

In [2]:
# Load Dataset
dataset_name = 'Sonar'
df = pd.read_csv('./datasets/sonar.csv')

In [3]:
# Scale
scaler = MinMaxScaler()
scaler.fit(df.values[:, :-1])
scaled_df = scaler.transform(df.values[:, :-1])
joblib.dump(scaler, f'models/{dataset_name}_scaler.pkl')

['models/Sonar_scaler.pkl']

In [4]:
# Get scaled bounds
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
np.savez(f'models/{dataset_name}_data_bounds.npz', lower_bound=lower_bound, upper_bound=upper_bound)

print(lower_bound, upper_bound)

0.0 1.0000000000000002


In [5]:
# Check if binary targets
df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
df_scaled['target'] = targets

Original Targets:  [0. 1.] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True


In [6]:
df_scaled

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f52,f53,f54,f55,f56,f57,f58,f59,f60,target
0,0.136431,0.156451,0.135677,0.035426,0.224956,0.237571,0.407468,0.340904,0.449282,0.285714,...,0.027104,0.155844,0.435673,0.149660,0.417949,0.502841,0.185355,0.245179,0.060046,1
1,0.323009,0.221603,0.272011,0.150024,0.283033,0.666756,0.574405,0.755458,0.483045,0.394537,...,0.108417,0.218182,0.111111,0.199546,0.479487,0.389205,0.105263,0.140496,0.087760,1
2,0.182153,0.246892,0.356110,0.243699,0.230028,0.585327,0.648810,0.819405,0.817859,0.869584,...,0.319544,0.418182,0.248538,0.394558,0.615385,0.889205,0.368421,0.258953,0.166282,1
3,0.062684,0.070724,0.199737,0.034950,0.034999,0.071486,0.288149,0.269239,0.077447,0.164593,...,0.161198,0.080519,0.409357,0.179138,0.176923,0.133523,0.093822,0.107438,0.256351,1
4,0.550885,0.282898,0.153088,0.079886,0.132640,0.147003,0.318182,0.531863,0.516659,0.621479,...,0.032810,0.127273,0.277778,0.235828,0.028205,0.196023,0.102975,0.292011,0.203233,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.126844,0.145735,0.050263,0.028293,0.082678,0.410642,0.539773,0.361411,0.333629,0.367653,...,0.154066,0.241558,0.552632,0.061224,0.248718,0.176136,0.256293,0.528926,0.348730,0
204,0.227139,0.040720,0.092970,0.120304,0.175755,0.230046,0.258929,0.212348,0.141419,0.291863,...,0.075606,0.228571,0.365497,0.129252,0.151282,0.088068,0.066362,0.168044,0.140878,0
205,0.373894,0.184741,0.054205,0.055635,0.072026,0.287288,0.331169,0.247630,0.175181,0.345488,...,0.216833,0.062338,0.119883,0.126984,0.217949,0.389205,0.308924,0.209366,0.057737,0
206,0.212389,0.148736,0.156045,0.130766,0.025361,0.336469,0.387446,0.235502,0.276914,0.320463,...,0.111270,0.106494,0.339181,0.068027,0.079487,0.088068,0.173913,0.096419,0.096998,0


In [7]:
# Train model with 25% of data
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

clf = svm.SVC(kernel='linear')

# Train the model using the training set
clf.fit(X_train, y_train)
joblib.dump(clf, f'models/{dataset_name}_svm_model.pkl')

# Predict the response for test dataset
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.86      0.82        83
           1       0.82      0.73      0.77        73

    accuracy                           0.79       156
   macro avg       0.80      0.79      0.79       156
weighted avg       0.80      0.79      0.79       156



In [8]:
X_test_df = pd.DataFrame(X_test)

# Save to CSV
X_test_df.to_csv(f'{dataset_name}_results/{dataset_name}_X_test.csv', index=False)

In [9]:
# Finding patterns classified as positive/negative
positive_indexes,negative_indexes = utility.find_indexes(clf, X_test, threshold=0)
print(f"Positive patterns = {len(positive_indexes)},\nNegative patterns = {len(negative_indexes)}")

Positive patterns = 65,
Negative patterns = 91


In [10]:
# Make a dataframe with the test data. For comparing Onestep against Twostep.
test_df_names = list(df.columns)
if 'target' not in test_df_names:
    test_df_names.append('target')
test_dataset = []
for instance, test_class in zip(X_test, y_test.astype('int32')):
    test_dataset.append(np.append(instance, test_class))
test_dataset_df = pd.DataFrame(np.asarray(test_dataset), columns=test_df_names)

In [11]:
test_dataset_df.to_csv(f'{dataset_name}_results/{dataset_name}_X_test_predicted.csv', index=False)

In [11]:
# Parameter p value
p = 0.50

#Variables for results
times_twostep = []
rsum_twostep = []
coverage_twostep = []
pos_exp_twostep = []
neg_exp_twostep = []

times_onestep = []
rsum_onestep = []
coverage_onestep = []
pos_exp_onestep = []
neg_exp_onestep = []

#### Onestep

In [12]:
#Generate Explanations for the patterns classified as negative
for idx in  negative_indexes:  
    #Onestep
    start = time.perf_counter()
    exp = docplex_explainer.onestep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            positive = False)
    end = time.perf_counter()
    times_onestep.append((end - start))
    neg_exp_onestep.append(exp)
    rsum_onestep.append(mymetrics.range_sum(exp))
    coverage_onestep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp)))

#Generate Explanations for the patterns classfied as positive
for idx in positive_indexes:
    #Onestep
    start = time.perf_counter()
    exp = docplex_explainer.onestep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            positive = True)
    end = time.perf_counter()
    times_onestep.append((end - start))
    pos_exp_onestep.append(exp)
    rsum_onestep.append(mymetrics.range_sum(exp))
    coverage_onestep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp)))

#### Twostep

In [13]:
#Generate Explanations for the patterns classified as negative
for idx in  negative_indexes:
    
    #Twostep
    start = time.perf_counter()
    exp_ = docplex_explainer.twostep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            p = p,
            positive = False)
    end = time.perf_counter()
    times_twostep.append((end - start))
    neg_exp_twostep.append(exp_)
    rsum_twostep.append(mymetrics.range_sum(exp_))
    coverage_twostep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp_)))


#Generate Explanations for the patterns classfied as positive
for idx in positive_indexes:
    
    #Twostep
    start = time.perf_counter()
    exp_ = docplex_explainer.twostep(
            classifier = clf,
            dual_coef = clf.dual_coef_,
            support_vectors = clf.support_vectors_,
            intercept = clf.intercept_,
            lower_bound = lower_bound,
            upper_bound = upper_bound,
            data = (X_test[idx]),
            p = p,
            positive = True)
    end = time.perf_counter()
    times_twostep.append((end - start))
    pos_exp_twostep.append(exp_)
    rsum_twostep.append(mymetrics.range_sum(exp_))
    coverage_twostep.append(len(mymetrics.calculate_coverage(test_dataset_df, exp_)))

#### Metrics

In [14]:
# Compute feature expansion sizes for Twostep
frequency_twostep_neg = utility.detail_exp(
    explanations=neg_exp_twostep, 
    patterns=X_test[negative_indexes],
    number_of_features=len(X_test[0]), 
    show_explanation=False, 
    show_frequency=False, 
    low_val=lower_bound, 
    upp_val=upper_bound
)

frequency_twostep_pos = utility.detail_exp(
    explanations=pos_exp_twostep, 
    patterns=X_test[positive_indexes],
    number_of_features=len(X_test[0]), 
    show_explanation=False, 
    show_frequency=False, 
    low_val=lower_bound, 
    upp_val=upper_bound
)

neg_sizes_twostep = np.count_nonzero(frequency_twostep_neg.to_numpy() == 1, axis=1)
pos_sizes_twostep = np.count_nonzero(frequency_twostep_pos.to_numpy() == 1, axis=1)
feature_sizes_twostep = np.concatenate([neg_sizes_twostep, pos_sizes_twostep])

In [15]:
# Compute feature expansion sizes for Onestep
frequency_onestep_neg = utility.detail_exp(
    explanations=neg_exp_onestep, 
    patterns=X_test[negative_indexes],
    number_of_features=len(X_test[0]), 
    show_explanation=False, 
    show_frequency=False, 
    low_val=lower_bound, 
    upp_val=upper_bound
)

frequency_onestep_pos = utility.detail_exp(
    explanations=pos_exp_onestep, 
    patterns=X_test[positive_indexes],
    number_of_features=len(X_test[0]), 
    show_explanation=False, 
    show_frequency=False, 
    low_val=lower_bound, 
    upp_val=upper_bound
)

# Use np.count_nonzero
neg_sizes_onestep = np.count_nonzero(frequency_onestep_neg.to_numpy() == 1, axis=1)
pos_sizes_onestep = np.count_nonzero(frequency_onestep_pos.to_numpy() == 1, axis=1)

# Concatenate directly
feature_sizes_onestep = np.concatenate([neg_sizes_onestep, pos_sizes_onestep])

In [16]:
# Compute means and standard deviations
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

# Compute relative percentage differences
def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100

# Ensure all lists are NumPy arrays
times_onestep = np.array(times_onestep)
times_twostep = np.array(times_twostep)
feature_sizes_onestep = np.array(feature_sizes_onestep)
feature_sizes_twostep = np.array(feature_sizes_twostep)
rsum_onestep = np.array(rsum_onestep)
rsum_twostep = np.array(rsum_twostep)
coverage_onestep = np.array(coverage_onestep)
coverage_twostep = np.array(coverage_twostep)

# Compute means and standard deviations
(time_mean_onestep, time_std_onestep) = compute_mean_std(times_onestep)
(time_mean_twostep, time_std_twostep) = compute_mean_std(times_twostep)

(sizes_mean_onestep, sizes_std_onestep) = compute_mean_std(feature_sizes_onestep)
(sizes_mean_twostep, sizes_std_twostep) = compute_mean_std(feature_sizes_twostep)

(rsum_mean_onestep, rsum_std_onestep) = compute_mean_std(rsum_onestep)
(rsum_mean_twostep, rsum_std_twostep) = compute_mean_std(rsum_twostep)

(coverage_mean_onestep, coverage_std_onestep) = compute_mean_std(coverage_onestep)
(coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(coverage_twostep)

# Compute relative percentage differences (Mean & Std)
time_mean_diff = relative_percentage_diff(time_mean_twostep, time_mean_onestep)
sizes_mean_diff = relative_percentage_diff(sizes_mean_twostep, sizes_mean_onestep)
rsum_mean_diff = relative_percentage_diff(rsum_mean_twostep, rsum_mean_onestep)
coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_onestep)

time_std_diff = relative_percentage_diff(time_std_twostep, time_std_onestep)
sizes_std_diff = relative_percentage_diff(sizes_std_twostep, sizes_std_onestep)
rsum_std_diff = relative_percentage_diff(rsum_std_twostep, rsum_std_onestep)
coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_onestep)

# Compute pointwise relative differences
time_relative_pointwise = relative_percentage_diff(times_twostep, times_onestep)
sizes_relative_pointwise = relative_percentage_diff(feature_sizes_twostep, feature_sizes_onestep)
rsum_relative_pointwise = relative_percentage_diff(rsum_twostep, rsum_onestep)
coverage_relative_pointwise = relative_percentage_diff(coverage_twostep, coverage_onestep)

# Compute pointwise means
time_relative_mean = np.mean(time_relative_pointwise) 
sizes_relative_mean = np.mean(sizes_relative_pointwise)
rsum_relative_mean = np.mean(rsum_relative_pointwise)
coverage_relative_mean = np.mean(coverage_relative_pointwise)

# Compute pointwise standard deviations
time_relative_std = np.std(time_relative_pointwise) 
sizes_relative_std = np.std(sizes_relative_pointwise)
rsum_relative_std = np.std(rsum_relative_pointwise)
coverage_relative_std = np.std(coverage_relative_pointwise)

# Organize Data
all_metrics_data = {
    'Metric': ['Time', 'Size', 'Ranges_Sum', 'Coverage'],
    'ONESTEP_MEAN': [time_mean_onestep, sizes_mean_onestep, rsum_mean_onestep, coverage_mean_onestep],
    'ONESTEP_STD': [time_std_onestep, sizes_std_onestep, rsum_std_onestep, coverage_std_onestep],
    'TWOSTEP_MEAN': [time_mean_twostep, sizes_mean_twostep, rsum_mean_twostep, coverage_mean_twostep],
    'TWOSTEP_STD': [time_std_twostep, sizes_std_twostep, rsum_std_twostep, coverage_std_twostep],
    'MEAN_DIFF_%': [time_mean_diff, sizes_mean_diff, rsum_mean_diff, coverage_mean_diff],
    'STD_DIFF_%': [time_std_diff, sizes_std_diff, rsum_std_diff, coverage_std_diff],
    'POINTWISE_MEAN_%': [time_relative_mean, sizes_relative_mean, rsum_relative_mean, coverage_relative_mean],
    'POINTWISE_STD_%': [time_relative_std, sizes_relative_std, rsum_relative_std, coverage_relative_std]
}
# Display and save
all_metrics_df = pd.DataFrame(all_metrics_data)
display(all_metrics_df)
all_metrics_df.to_csv(f'{dataset_name}_results/results_{p}.csv', index=False)

#Save Raw Metric Data
raw_df = pd.DataFrame({
    "times_onestep": times_onestep, 
    "times_twostep": times_twostep,
    "feature_sizes_onestep": feature_sizes_onestep, 
    "feature_sizes_twostep": feature_sizes_twostep,
    "rsum_onestep": rsum_onestep, 
    "rsum_twostep": rsum_twostep,
    "coverage_onestep": coverage_onestep, 
    "coverage_twostep": coverage_twostep,
    "time_relative_%": time_relative_pointwise,
    "sizes_relative_%": sizes_relative_pointwise,
    "rsum_relative_%": rsum_relative_pointwise,
    "coverage_relative_%": coverage_relative_pointwise
})

# Save to CSV
raw_df.to_csv(f"{dataset_name}_results/raw_metric_data_{p}.csv", index=False)

np.savez(f'{dataset_name}_results/neg_explanations_{p}.npz', 
         neg_exp_onestep=neg_exp_onestep, 
         neg_exp_twostep=neg_exp_twostep)

# Save positive explanations
np.savez(f'{dataset_name}_results/pos_explanations_{p}.npz', 
         pos_exp_onestep=pos_exp_onestep, 
         pos_exp_twostep=pos_exp_twostep)

C:\Users\Mateus\AppData\Local\Temp\ipykernel_428\760104910.py:9: RuntimeWarning: invalid value encountered in scalar divide
  return np.where(old != 0, ((new - old) / old) * 100, np.nan)


,Metric,ONESTEP_MEAN,ONESTEP_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%
0,Time,0.183423,0.010009,0.300317,0.026482,63.728761,164.582239,63.782587,12.107260
1,Size,51.647436,3.800817,51.647436,3.800817,0.000000,0.0,0.000000,0.000000
2,Ranges_Sum,31.849594,2.596340,31.854303,2.597341,0.014787,0.038559,0.014639,0.015894
3,Coverage,1.000000,0.000000,1.000000,0.000000,0.000000,nan,0.000000,0.000000
